In [2]:
import numpy as np
from plyfile import PlyData, PlyElement
import os
import pandas as pd

fp = "/media/hristo/sharedData1/00_Projects/24-10-26_SPP_PCS/02_Datasets/FWF_Aachen_labeled/2024-03-22_FW_Koenigshuegel.FwfProj/labeled/2024-03-22_FW_Koenigshuegel_pointcloud.ply"
pcd = pd.DataFrame.from_dict(PlyData.read(fp).elements[0].data)
xyz = pcd[['x','y','z']].to_numpy()


In [3]:
import numpy as np
from scipy.spatial import KDTree

def calculate_normals(points, k=20):
    # Create a KDTree for the points
    tree = KDTree(points)
    
    normals = np.zeros_like(points)  # Array to store the normals
    
    for i, point in enumerate(points):
        # Find the k-nearest neighbors of the point
        distances, indices = tree.query(point, k=k+1)  # k+1 because the point itself is included
        neighbors = points[indices[1:]]  # Exclude the point itself
        
        # Compute the covariance matrix of the neighbors
        cov_matrix = np.cov(neighbors - point, rowvar=False)
        
        # Compute eigenvalues and eigenvectors
        eigenvalues, eigenvectors = np.linalg.eigh(cov_matrix)
        
        # The normal is the eigenvector corresponding to the smallest eigenvalue
        normal = eigenvectors[:, 0]
        normals[i] = normal
    
    return normals

# Example usage
points = np.random.rand(1000, 3)  # Example point cloud with 1000 points
normals = calculate_normals(points, k=20)


In [26]:
kd_tree = KDTree(xyz)

In [27]:
neibs = kd_tree.query(xyz,20)

In [18]:
import numpy as np
from scipy.spatial import KDTree

def calculate_normals_vectorized_v1(points, k=20):
    # Create a KDTree for efficient nearest-neighbor search
    tree = KDTree(points)
    
    # Find k-nearest neighbors for all points
    _, indices = tree.query(points, k=k+1)  # k+1 because the point itself is included
    
    # Gather neighbors for all points
    neighbors = points[indices[:, 1:]]  # Exclude the point itself (Shape: (N, k, 3))
    
    # Compute the mean of neighbors
    neighbors_mean = neighbors.mean(axis=1, keepdims=True)  # Shape: (N, 1, 3)
    
    # Center the neighbors by subtracting the mean
    centered_neighbors = neighbors - neighbors_mean  # Shape: (N, k, 3)
    
    # Compute covariance matrices for all points
    cov_matrices = np.einsum('nik,nij->nkj', centered_neighbors, centered_neighbors) / (k - 1)
    
    # Perform eigenvalue decomposition to get eigenvectors
    eigenvalues, eigenvectors = np.linalg.eigh(cov_matrices)  # Shapes: (N, 3), (N, 3, 3)
    
    # Extract the normal vector (eigenvector corresponding to the smallest eigenvalue)
    normals = eigenvectors[:, :, 0]  # Shape: (N, 3)
    
    return normals

# # Example usage
# points = np.random.rand(1500, 3)  # Example point cloud with 1266756 points
# normals = calculate_normals_vectorized(points, k=20)

# print(normals.shape)  # Should print (1266756, 3)


In [19]:
import numpy as np
from scipy.spatial import KDTree

def calculate_normals_vectorized_v2(points, k=20):
    # Create a KDTree for efficient nearest-neighbor search
    tree = KDTree(points)

    # Find k-nearest neighbors for all points
    _, indices = tree.query(points, k=k+1)  # k+1 because the point itself is included

    # Gather neighbors for all points
    neighbors = points[indices[:, 1:]]  # Exclude the point itself (first neighbor)

    # Compute the mean of neighbors (for covariance computation)
    neighbors_mean = neighbors.mean(axis=1, keepdims=True)  # Shape: (N, 1, 3)

    # Center the neighbors by subtracting the mean
    centered_neighbors = neighbors - neighbors_mean  # Shape: (N, k, 3)

    # Compute covariance matrices for all points
    cov_matrices = np.einsum('nki,nkj->nij', centered_neighbors, centered_neighbors) / (k - 1)
    # Alternatively, you can use np.matmul:
    # cov_matrices = np.matmul(centered_neighbors.transpose(0, 2, 1), centered_neighbors) / (k - 1)

    # Perform eigenvalue decomposition to get eigenvectors
    eigenvalues, eigenvectors = np.linalg.eigh(cov_matrices)  # Shapes: (N, 3), (N, 3, 3)

    # The normal is the eigenvector corresponding to the smallest eigenvalue
    normals = eigenvectors[:, :, 0]  # Shape: (N, 3)

    # Ensure normals are unit vectors
    normals = normals / np.linalg.norm(normals, axis=1, keepdims=True)

    return normals

# # Example usage
# points = np.random.rand(1500, 3)  # Example point cloud with 1266756 points
# normals = calculate_normals_vectorized(points, k=20)

# print(normals.shape)  # Should print (1266756, 3)


In [13]:
centered_neighbors.shape

(1266756, 20, 3)

In [11]:
points = xyz
k = 20
# Create a KDTree for efficient nearest-neighbor search
tree = KDTree(points)

# Find k-nearest neighbors for all points
_, indices = tree.query(points, k=k+1)  # k+1 because the point itself is included

# Gather neighbors for all points
neighbors = points[indices[:, 1:]]  # Exclude the point itself (first neighbor)

# Compute the mean of neighbors (for covariance computation)
neighbors_mean = neighbors.mean(axis=1, keepdims=True)  # Shape: (N, 1, 3)

# Center the neighbors by subtracting the mean
centered_neighbors = neighbors - neighbors_mean  # Shape: (N, k, 3)

# Compute covariance matrices for all points
cov_matrices = np.einsum('nij,nkj->nik', centered_neighbors, centered_neighbors) / (k - 1)

# Perform eigenvalue decomposition to get eigenvectors
eigenvalues, eigenvectors = np.linalg.eigh(cov_matrices)  # Shape: (N, 3), (N, 3, 3)

# The normal is the eigenvector corresponding to the smallest eigenvalue
normals = eigenvectors[:, :, 0]  # Take the first eigenvector (smallest eigenvalue)



In [4]:
norms = calculate_normals(xyz)

In [22]:
norms = calculate_normals_vectorized_v1(xyz)

In [10]:
norms.shape

(1266756, 20)

In [23]:
npcd = PlyData([PlyElement.describe(pd.DataFrame(np.concatenate([xyz, norms],axis=1),columns=['x','y','z','nx','ny','nz']).to_records(index=False),'vertex')]).write("./_temp/newimplement_knn_vectorized_v1.ply")